# STEDI Data Exploration (Bronze Layer)

This notebook explores the STEDI step test data using SQL and PySpark.
The goal is to understand the raw data structure, perform basic cleaning,
and prepare features that will be reused later in the ML pipeline.

In [0]:
-- How many rows do we have?
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows
FROM workspace.bronze.device_messages_raw
UNION ALL
SELECT 'rapid_step_tests_raw', COUNT(*)
FROM workspace.bronze.rapid_step_tests_raw;

This query checks the number of records in each bronze table to confirm
that the data was successfully ingested into the catalog.

This query groups raw device messages by device_id to identify which devices are sending the highest number of messages. This helps validate ingestion and detect potential outliers or noisy devices.

In [0]:
-- Count messages per device (top 10)
SELECT
  device_id,
  COUNT(*) AS messages
FROM workspace.bronze.device_messages_raw
GROUP BY device_id
ORDER BY messages DESC
LIMIT 10;

This query shows which devices are generating the most sensor messages.
This helps identify high-activity devices and potential outliers.

In [0]:
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  device_id,
  TRY_CAST(
    REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '')
    AS DOUBLE
  ) AS distance_cm,
  CAST(timestamp AS BIGINT) AS ts_ms,
  CAST(date AS BIGINT) AS date_ms,
  message_origin,
  sensor_type,
  message
FROM workspace.bronze.device_messages_raw;

In [0]:
SELECT * FROM device_messages_clean LIMIT 20;

Distance values may appear as strings (for example, "1cm").
This step removes non-numeric characters and safely casts distance to centimeters.

In [0]:
SELECT device_id,
       AVG(distance_cm) AS avg_cm,
       MIN(distance_cm) AS min_cm,
       MAX(distance_cm) AS max_cm
FROM device_messages_clean
GROUP BY device_id
ORDER BY avg_cm DESC
LIMIT 10;

In [0]:
WITH exploded AS (
  SELECT
    customer,
    device_id,
    start_time,
    posexplode(step_points) AS (step_index, step_ms)
  FROM workspace.bronze.rapid_step_tests_raw
)
SELECT customer, device_id,
       COUNT(*) AS steps,
       AVG(step_ms) AS avg_step_ms,
       STDDEV(step_ms) AS sd_step_ms
FROM exploded
GROUP BY customer, device_id
ORDER BY steps DESC
LIMIT 10;

In [0]:
WITH tests AS (
  SELECT customer, device_id, start_time, stop_time, test_time, total_steps
  FROM workspace.bronze.rapid_step_tests_raw
),
msgs AS (
  SELECT device_id, ts_ms, distance_cm
  FROM device_messages_clean
  WHERE distance_cm IS NOT NULL
)
SELECT
  t.customer,
  t.device_id,
  t.start_time,
  t.stop_time,
  COUNT(m.ts_ms) AS readings_in_window,
  AVG(m.distance_cm) AS avg_cm_in_window,
  MIN(m.distance_cm) AS min_cm_in_window,
  MAX(m.distance_cm) AS max_cm_in_window
FROM tests t
JOIN msgs m
  ON m.device_id = t.device_id
 AND m.ts_ms BETWEEN t.start_time AND t.stop_time
GROUP BY t.customer, t.device_id, t.start_time, t.stop_time
ORDER BY readings_in_window DESC
LIMIT 20;

This query shows how many sensor readings occurred during each test
and the average, minimum, and maximum distance within that test window.
These aggregates will later be reused as features for machine learning.


In [0]:
%python
dm = spark.table("workspace.bronze.device_messages_raw")
rt = spark.table("workspace.bronze.rapid_step_tests_raw")

dm.printSchema()
rt.printSchema()


In [0]:
%python
from pyspark.sql import functions as F

dm_clean = (
    dm
    .withColumn("distance_str", F.col("distance").cast("string"))
    .withColumn("distance_cm", F.regexp_replace("distance_str", "[^0-9.]", "").cast("double"))
    .withColumn("ts_ms", F.col("timestamp").cast("bigint"))
    .withColumn("date_ms", F.col("date").cast("bigint"))
)

display(dm_clean.limit(10))

In [0]:
%python
dm_stats = (
    dm_clean
    .groupBy("device_id")
    .agg(
        F.count("*").alias("n"),
        F.avg("distance_cm").alias("avg_cm"),
        F.min("distance_cm").alias("min_cm"),
        F.max("distance_cm").alias("max_cm")
    )
    .orderBy(F.desc("n"))
)

display(dm_stats.limit(20))


In [0]:
%python
rt_exploded = (
    rt
    .select(
        "customer", "device_id", "start_time", "stop_time",
        "test_time", "total_steps",
        F.posexplode("step_points").alias("step_index", "step_ms")
    )
)

display(rt_exploded.limit(20))


In [0]:
%python
msgs = dm_clean.select("device_id", "ts_ms", "distance_cm") \
               .where(F.col("distance_cm").isNotNull())

tests = rt.select("customer", "device_id", "start_time", "stop_time", "test_time", "total_steps")

joined = (
    tests.alias("t")
    .join(
        msgs.alias("m"),
        (F.col("m.device_id") == F.col("t.device_id")) &
        (F.col("m.ts_ms").between(F.col("t.start_time"), F.col("t.stop_time"))),
        how="inner"
    )
)

features = (
    joined
    .groupBy("t.customer", "t.device_id", "t.start_time", "t.stop_time", "t.test_time", "t.total_steps")
    .agg(
        F.count("m.ts_ms").alias("readings_in_window"),
        F.avg("m.distance_cm").alias("avg_cm_in_window"),
        F.min("m.distance_cm").alias("min_cm_in_window"),
        F.max("m.distance_cm").alias("max_cm_in_window"),
        F.var_pop("m.distance_cm").alias("var_cm_in_window")
    )
    .orderBy(F.desc("readings_in_window"))
)

display(features.limit(20))


Cleaning and querying the STEDI data was straightforward once the schemas were understood,
but time-based joins required careful attention to timestamps. One ethics risk is incorrect
data labeling or casting (such as distance values), which could lead to misleading features
and harmful conclusions in health-related models. Honesty and accuracy in data preparation
reflect our responsibility as disciples to seek truth.